In [0]:
%sql

----------------- Add sourceoftruth to default.viw_additionalfields ----------------

ALTER VIEW default.viw_additionalfields AS
SELECT DISTINCT
  rd.visualrequestfilenumber AS `Request ID`,
  frccf.identityverification,
  frccf.previousfoirequest,
  frccf.physicalpageestimate,
  frccf.electronicpageestimate,
  frccf.noofpagesdeduplicated,
  frccf.applicationfeepaid,
  frccf.applicationfeerefunded,
  frccf.applicationfeetransferred,
  frccf.linkedrequests,
  eco.officecode AS `Action Office`,
  rd.closeddate AS `Closed Date`,
  rd.targetdate AS `Due Date`,
  rd.foirequestid,
  rd.officeid,
  rd.visualrequestfilenumber,
  rd.amendmentcreateddate,
  rd.refvisualrequestfilenumber,
  rd.closedby,
  rd.closeddate,
  rd.notes,
  rd.receivedbyusername,
  rd.createddate,
  rd.currentactivity,
  rd.deliverydate,
  rd.targetdate,
  rd.disposition,
  rd.originalcloseddate,
  rd.originaltargetdate,
  rd.originalreceiveddate,
  rd.primaryusername,
  rd.processeddays,
  rd.receiveddate,
  rd.referencenumber,
  rd.remainingdays,
  rd.requestdescription,
  rd.caseowner,
  rd.status,
  rd.startdate,
  rd.shipaddressid,
  rd.billaddressid,
  rd.receivedmodeid,
  rd.deliverymodeid,
  rd.primarygroupid,
  rd.requesttypeid,
  rd.onbehalfofrequesterid,
  rd.sourceoftruth
FROM
  factRequestDetails rd
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid
WHERE
  rd.activeflag = 'Y'

In [0]:
%sql

----------------- Update LEFT JOIN dimRequesters req to default.viw_sampletlreport ----------------

ALTER VIEW default.viw_sampletlreport AS
SELECT
  rd.visualrequestfilenumber AS Request_ID,
  rt.requesttypename AS Request_Type,
  category.requestertypename AS Applicant_Category,
  rd.redactiondescription AS Request_Description,
  rd.startdate AS Start_Date,
  rd.targetdate AS Due_Date,
  rd.remainingdays AS Remaining_Days,
  rd.currentactivity AS Current_Action,
  rd.requeststatus AS Request_Status,
  rd.primaryusername AS Primary_User,
  rd.execcomments AS Comments,
  rd.extension AS Number_of_Extensions_Applied,
  lastinvoice.invoiceddate AS Last_Invoice_Date,
  rdd.noofpagesintherequest AS Total_pages_in_request,
  eco.officedescription AS Office_Name,
  eco.officecode AS Office_Code,
  rd.caseowner AS Request_Owner,
  CONCAT(req.lastname, ', ', req.firstname) AS Applicant_Name,
  rd.groupname AS User_Group,
  rd.requesteddate AS Requested_Date,
  rd.receiveddate AS Received_Date,
  rd.closeddate AS Closed_Date,
  rd.originalcloseddate AS Original_Closed_Date,
  rd.requestage AS Request_Age
FROM
  factRequestDetails rd
    LEFT JOIN dimRequests r
      ON (
        rd.foirequestid = r.foirequestid
        and rd.sourceoftruth = r.sourceoftruth
      )
    LEFT JOIN dimRequestTypes rt
      ON rt.requesttypeid = rd.requesttypeid
    LEFT JOIN (
      SELECT
        foirequestid,
        invoiceddate,
        ROW_NUMBER() OVER (PARTITION BY foirequestid ORDER BY createddate DESC) AS rn
      FROM
        factRequestInvoices
      WHERE
        activeflag = 'Y'
    ) lastinvoice
      ON lastinvoice.foirequestid = rd.foirequestid
      AND lastinvoice.rn = 1
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rdd.foirequestid = rd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN dimECOffice eco
      ON eco.officeid = rd.officeid
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes category
      ON rr.requestertypeid = category.requestertypeid
    LEFT JOIN dimRequesters req
      ON
        ((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rr.sourceoftruth IS NULL OR rr.sourceoftruth != 'FOIMOD') AND rr.requesterid = req.requesterid)
        OR
        (req.sourceoftruth = 'FOIMOD' AND rr.sourceoftruth = 'FOIMOD' AND req.foirequestid = rr.foirequestid AND req.requesterid = rr.requesterid)
WHERE
  rd.activeflag = 'Y'

In [0]:
%sql

----------------- Update LEFT JOIN dimRequesters req to default.viw_analystworkloaddash

ALTER VIEW default.viw_analystworkloaddash AS
SELECT DISTINCT
  rd.foirequestid,
  rd.visualrequestfilenumber AS requestid,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN 'Intake'
    ELSE ecggn.groupname
  END AS team,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN ecg.groupdescription
    ELSE ecggn.groupdescription
  END AS manager,
  rd.ministry,
  eco.officecode AS procorg,
  rd.applicantcategoryid AS category,
  rt.requesttypename AS type,
  rqt.requestertypename AS applicanttype,
  concat(req.lastname, ', ', req.firstname) AS applicantname,
  frccf.subject,
  rd.startdate,
  rd.closeddate AS enddate,
  rd.targetdate AS duedate,
  rd.processeddays AS totalprocesseddays,
  rd.currentactivity,
  frccf.currentactivitydate,
  rd.primaryusername AS analyst,
  frccf.crossgovtno,
  rd.requestdescription AS description,
  rd.status,
  frccf.customfieldstatus AS `Custom Status`,
  frccf.countontime,
  frccf.countoverdue,
  frccf.onholddays,
  frccf.daysoverdue,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    ELSE 'Not Closed'
  END AS notclosed,
  rpt.estimatedamount AS feesest,
  rpt.incurredamount AS feeswaived,
  rpt.paidamount AS feespaid,
  rd.disposition,
  frccf.publication,
  frccf.publicationreason,
  CASE
    WHEN rd.extension IS NULL THEN 'N'
    ELSE 'Y'
  END AS extension,
  rd.execcomments,
  frccf.noofpagesreleased AS noofpagesdelivered,
  rdd.noofpagesintherequest,
  CASE
    WHEN
      month(rd.receiveddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.receiveddate), year(add_months(rd.receiveddate, 12)))
    WHEN month(rd.receiveddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.receiveddate, -12)), year(rd.receiveddate))
  END AS startfyr,
  CASE
    WHEN
      month(rd.closeddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.closeddate), year(add_months(rd.closeddate, 12)))
    WHEN month(rd.closeddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.closeddate, -12)), year(rd.closeddate))
  END AS endfyr,
  CASE
    WHEN month(rd.receiveddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.receiveddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.receiveddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.receiveddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS startfqtr,
  CASE
    WHEN month(rd.closeddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.closeddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.closeddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.closeddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS endfqtr,
  d.division,
  s.name AS stage,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    WHEN frccf.countontime = 1 THEN 'Active'
    WHEN frccf.countoverdue = 1 THEN 'Overdue'
    ELSE NULL
  END AS overduestatus,
  ffrs.name AS foiflowstatus,
  ffrsd.ministryassigneefullname AS assignedministryperson,
  rds2.divisions,
  rd.requeststatus,
  month(rd.startdate) AS startmonth,
  rd.requesttypeid,
  COALESCE(CAST(ffrsd.recordspagecount AS INT), 0) AS recordspagecount,
  COALESCE(CAST(ffrsd.lanpagecount AS INT), 0) AS lanpagecount
FROM
  factRequestDetails rd
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid
    LEFT JOIN dimECGroups ecggn
      ON rd.groupname = ecggn.groupname
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    LEFT JOIN factRequestPaymentTransaction rpt
      ON rd.foirequestid = rpt.foirequestid
      AND rpt.activeflag = 'Y'
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rd.foirequestid = rdd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes rqt
      ON rd.applicantcategoryid = rqt.requestertypeid
    LEFT JOIN dimRequesters req
      ON
        ((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.requesterid = req.requesterid)
        OR
        (req.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid)
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factRequestDivisionalStages
      GROUP BY
        foirequestnumber
    ) urds
      ON urds.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestDivisionalStages rds
      ON rds.foirequestnumber = rd.visualrequestfilenumber
      AND urds.runcycleid = rds.runcycleid
      AND urds.createddate = rds.createddate
    LEFT JOIN dimStages s
      ON s.stageid = rds.stageid
    LEFT JOIN dimDivisions d
      ON d.divisionid = rds.divisionid
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factFOIFlowRequestStatusDetails
      GROUP BY
        foirequestnumber
    ) uffrsd
      ON uffrsd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factFOIFlowRequestStatusDetails ffrsd
      ON ffrsd.foirequestnumber = rd.visualrequestfilenumber
      AND uffrsd.runcycleid = ffrsd.runcycleid
      AND uffrsd.createddate = ffrsd.createddate
    LEFT JOIN dimFOIFlowRequestStatus ffrs
      ON ffrs.requeststatusid = ffrsd.requeststatusid
    LEFT JOIN (
      SELECT
        rds_1.foirequestnumber,
        string_agg(CONCAT('<p>', d_1.division, ' - ', s_1.name, '</p>'), '\n') AS divisions
      FROM
        (
          SELECT
            foirequestnumber,
            MAX(foirequestversionid) AS foirequestversionid,
            MAX(runcycleid) AS runcycleid
          FROM
            factRequestDivisionalStages
          GROUP BY
            foirequestnumber
        ) sq
          JOIN factRequestDivisionalStages rds_1
            ON rds_1.foirequestnumber = sq.foirequestnumber
            AND rds_1.foirequestversionid = sq.foirequestversionid
            AND rds_1.runcycleid = sq.runcycleid
          JOIN dimStages s_1
            ON s_1.stageid = rds_1.stageid
          JOIN dimDivisions d_1
            ON d_1.divisionid = rds_1.divisionid
      GROUP BY
        rds_1.foirequestnumber
    ) rds2
      ON rds2.foirequestnumber = rd.visualrequestfilenumber
WHERE
  rd.activeflag = 'Y'
  AND rd.officeid NOT IN (1, 10, 21, 33, 41, 45, 49, 51, 55)
  AND rd.requesttypeid IN (31, 33)

In [0]:
%sql

----------------- Update LEFT JOIN dimRequesters req to default.viw_allopenclosed ----------------

ALTER VIEW default.viw_allopenclosed AS
SELECT DISTINCT
  rd.visualrequestfilenumber AS `Request ID`,
  rd.groupname AS `User Group`,
  rd.caseowner AS `Request Owner`,
  rd.ministry AS `Public Body`,
  eco.officecode AS `Office Code`,
  rt.requesttypename AS `Request Type`,
  category.requestertypename AS `Applicant Category`,
  req.fullname AS `Applicant Name`,
  rd.subject AS `Subject`,
  rd.startdate AS `Start Date`,
  rd.targetdate AS `Due Date`,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN rd.closeddate
    ELSE NULL
  END AS `Closed Date`,
  rd.processeddays AS `Total Process Days`,
  rd.currentactivity AS `Current Action`,
  rd.primaryusername AS `Primary User`,
  rd.crossgovtno AS `Cross Gov #`,
  rd.requestdescription AS `Description`,
  rs.requeststatusname AS `Request Status`,
  custom.countontime AS `Count On Time`,
  custom.countoverdue AS `Count Overdue`,
  custom.onholddays AS `On Hold Days`,
  custom.daysoverdue AS `Days Overdue`,
  pt.estimatedamount AS `Fees Est`,
  pt.feewaived AS `Fees Waived`,
  custom.feepaidamount AS `Fees Paid`,
  rd.disposition AS `Disposition`,
  rd.publication AS `Publication`,
  rd.publicationreason AS `Publication Reason`,
  ext.numberofextensions AS `Extension`,
  rdd.noofpagesintherequest AS `Total # of Pages`,
  frd.recordspagecount AS `Total Mod Pages`
FROM
  factRequestDetails rd
    LEFT JOIN dimRequests r
      ON (
        rd.foirequestid = r.foirequestid
        and rd.sourceoftruth = r.sourceoftruth
      )
      AND r.cactive = 'Y'
    LEFT JOIN dimRequestTypes rt
      ON rt.requesttypeid = rd.requesttypeid
      AND rt.cactive = 'Y'
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rdd.foirequestid = rd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN dimECOffice eco
      ON eco.officeid = rd.officeid
    LEFT JOIN dimRequestStatuses rs
      ON rd.requeststatus = rs.requeststatus
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes category
      ON rd.applicantcategoryid = category.requestertypeid
    LEFT JOIN dimRequesters req
      ON
		((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.requesterid = req.requesterid)
		OR
		(req.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid)
    LEFT JOIN factRequestCustomCalcFields custom
      ON rd.foirequestid = custom.foirequestid
    LEFT JOIN factRequestInvoices ri
      ON rd.foirequestid = ri.foirequestid
      AND ri.activeflag = 'Y'
    LEFT JOIN dimMODMinistryRequestIDs rid
      ON rd.foirequestid = rid.foirequestid
    LEFT JOIN (
      SELECT
        foirequestid,
        COUNT(DISTINCT requestextid) AS numberofextensions
      FROM
        factRequestExtensions
      WHERE
        approvedstatus = 'A'
        AND activeflag = 'Y'
      GROUP BY
        foirequestid
    ) ext
      ON rd.foirequestid = ext.foirequestid
    LEFT JOIN (
      SELECT DISTINCT
        a.foirequestnumber,
        a.recordspagecount
      FROM
        factFOIFlowRequestStatusDetails a
          JOIN (
            SELECT
              foirequestnumber,
              MAX(version) AS maxv
            FROM
              factFOIFlowRequestStatusDetails
            WHERE
              recordspagecount IS NOT NULL
              AND isactive = TRUE
            GROUP BY
              foirequestnumber
          ) c
            ON c.foirequestnumber = a.foirequestnumber
            AND c.maxv = a.version
      WHERE
        a.recordspagecount IS NOT NULL
        AND a.isactive = TRUE
    ) frd
      ON frd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestPaymentTransaction pt
      ON rd.foirequestid = pt.foirequestid
      AND pt.activeflag = 'Y'
    JOIN (
      SELECT
        MAX(runcycleid) AS maxruncycleid,
        visualrequestfilenumber
      FROM
        factRequestDetails
      WHERE
        activeflag = 'Y'
      GROUP BY
        visualrequestfilenumber
    ) maxruncycle
      ON rd.visualrequestfilenumber = maxruncycle.visualrequestfilenumber
      AND rd.runcycleid = maxruncycle.maxruncycleid
WHERE
  rd.activeflag = 'Y'
  AND (
    rd.closeddate >= TIMESTAMP('2019-04-01 00:00:00')
    OR rd.closeddate = TIMESTAMP('1900-01-01 00:00:00')
  )
  AND eco.officecode NOT IN ('IAO', 'IMB', 'XGR', 'PROAC')

In [0]:
%sql

----------------- Update default.viw_broaderaxisfields ----------------
----------------- LEFT JOIN dimAddress aship
----------------- LEFT JOIN dimAddress abill
----------------- LEFT JOIN dimRequesters rqstr
----------------- LEFT JOIN dimRequesters onbehalfofrequesterid

ALTER VIEW default.viw_broaderaxisfields AS
SELECT
  rd.visualrequestfilenumber AS `Request ID`,
  re.extensiondays AS `# of days extended`,
  frccf.noofdocdelivered AS `# of documents delivered`,
  frccf.noofpagesreleased AS `# of pages released`,
  frccf.noofpagesreviewed AS `# of pages reviewed`,
  eco.officecode AS `Action Office`,
  rd.amendmentcreateddate AS `Amended Date`,
  rpt.estimatedamount AS `Amount Estimated ($)`,
  rpt.paidamount AS `Amount Paid ($)`,
  concat_ws(
    ', ', abill.address1, abill.address2, abill.city, abill.state, abill.country, abill.zipcode
  ) AS `Applicant Billing Address`,
  rqstrt.requestertypename AS `Applicant Category`,
  rqstr.email AS `Applicant Email`,
  rqstr.fax AS `Applicant Fax`,
  rpt.applicationfee AS `Application Fee ($)`,
  rd.refvisualrequestfilenumber AS `Applicant's File Reference`,
  rqstr.home AS `Applicant Home Phone`,
  rqstr.mobile AS `Applicant Mobile`,
  rqstr.fullname AS `Applicant Name`,
  concat_ws(
    ', ', aship.address1, aship.address2, aship.city, aship.state, aship.country, aship.zipcode
  ) AS `Applicant Shipping Address`,
  rqstr.workphone1 AS `Applicant Work Phone1`,
  rqstr.workphone2 AS `Applicant Work Phone2`,
  rpt.invoicedamount - rpt.paidamount AS `Balance Amount ($)`,
  rd.closedby AS `Closed By`,
  rd.closeddate AS `Closed Date`,
  rd.notes AS `Comments`,
  frccf.coordinatednrresponsereqd AS `Coordinated NR Response Req'd`,
  rd.receivedbyusername AS `Created By`,
  rd.createddate AS `Created Date`,
  rd.currentactivity AS `Current Action`,
  rd.deliverydate AS `Delivery Date`,
  dm.deliverymodename AS `Delivery Mode`,
  rd.targetdate AS `Due Date`,
  re.approveddate AS `Extension Approval/Denial Date`,
  re.approvedcomments AS `Extension Approval/Denial Note`,
  re.completeddate AS `Extension Completed Date`,
  re.completedcomments AS `Extension Completion Note`,
  re.comments AS `Extension Note`,
  et.extensiontypename AS `Extension Reason`,
  CASE
    WHEN
      re.cstatus = 'C'
      AND re.approvedstatus = 'A'
    THEN
      'Completed (Approved)'
    WHEN
      re.cstatus = 'P'
      AND re.approvedstatus = 'N'
    THEN
      'Pending'
    WHEN
      re.cstatus = 'A'
      AND re.approvedstatus = 'A'
    THEN
      'Approved'
    WHEN
      re.cstatus = 'D'
      AND re.approvedstatus = 'D'
    THEN
      'Denied'
    WHEN
      re.cstatus = 'C'
      AND re.approvedstatus = 'D'
    THEN
      'Completed (Denied)'
    ELSE NULL
  END AS `Extension Status`,
  rpt.feewaivedescription AS `Fee Waiver Request Reason`,
  rpt.feewaiverequested AS `Fee Waiver Requested`,
  rpt.feewaivegranted AS `Fee Waiver Status`,
  rd.disposition AS `Final Disposition`,
  rpt.invoicedamount AS `Last Invoice Amount ($)`,
  rpt.invoicedate AS `Last Invoice Date`,
  rpt.invoicenumber AS `Last Invoice Number`,
  frccf.applicantfilereference AS `Link Requests`,
  pm.paymentmodename AS `Method of Payment`,
  frccf.noofpagesdeduplicated AS `Number of Pages Deduplicated`,
  onbehalfofrequesterid.fullname AS `On Behalf Of`,
  rqstr.company AS `Organization`,
  rd.originalcloseddate AS `Original Closed Date`,
  rd.originaltargetdate AS `Original Due Date`,
  rd.originalreceiveddate AS `Original Start Date`,
  rpt.paymentreceiveddate AS `Payment Date`,
  rpt.paymentstatus AS `Payment Status`,
  rd.primaryusername AS `Primary User`,
  ecg.groupname AS `Primary User Group`,
  rd.processeddays AS `Processed Days`,
  rd.receiveddate AS `Received Date`,
  rm.receivedmodename AS `Received Mode`,
  rd.referencenumber AS `Reference Number`,
  frccf.refundamount AS `Refund Amount`,
  rd.remainingdays AS `Remaining Days`,
  rd.requestdescription AS `Request Description`,
  rd.caseowner AS `Request Owner`,
  rd.status AS `Request Status`,
  rt.requesttypename AS `Request Type`,
  frccf.secondaryusers AS `Secondary Users`,
  rd.startdate AS `Start Date`,
  frccf.subject AS `Subject`,
  rdd.noofpagesintherequest AS `Total # of pages in a request`,
  rdd.noofpagesinreviewlog AS `Total # of pages in the review log`,
  frccf.onholddays AS `Total Days on Hold`,
  rpt.totalinvoicedamount AS `Total Invoiced ($)`,
  frccf.crossgovtno AS `XGOV`,
  COALESCE(ri.chargedamount, 0) AS `Charged Amount ($)`
FROM
  viw_additionalfields rd
    LEFT JOIN dimRequests r
      ON rd.foirequestid = r.foirequestid
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN dimAddress aship
      ON
        ((aship.sourceoftruth IS NULL OR aship.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.shipaddressid = aship.addressid)
        OR
        (aship.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND aship.foirequestid = rd.foirequestid)
    LEFT JOIN dimAddress abill
      ON
        ((abill.sourceoftruth IS NULL OR abill.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.shipaddressid = abill.addressid)
        OR
        (abill.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND abill.foirequestid = rd.foirequestid)
    LEFT JOIN factRequestExtensions re
      ON rd.foirequestid = re.foirequestid
      AND re.activeflag = 'Y'
    LEFT JOIN dimExtensionTypes et
      ON re.extensiontypeid = et.extensiontypeid
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesters rqstr
      ON
        ((rqstr.sourceoftruth IS NULL OR rqstr.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rr.requesterid = rqstr.requesterid)
        OR
        (rqstr.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND rqstr.foirequestid = rd.foirequestid AND rqstr.requesterid = rr.requesterid)
    LEFT JOIN dimReceivedModes rm
      ON rd.receivedmodeid = rm.receivedmodeid
    LEFT JOIN dimDeliveryModes dm
      ON rd.deliverymodeid = dm.deliverymodeid
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rd.foirequestid = rdd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN factRequestPaymentTransaction rpt
      ON rd.foirequestid = rpt.foirequestid
      AND rpt.activeflag = 'Y'
    LEFT JOIN dimPaymentModes pm
      ON rpt.paymentmodeid = pm.paymentmodeid
    LEFT JOIN dimRequesters onbehalfofrequesterid
      ON
        ((onbehalfofrequesterid.sourceoftruth IS NULL OR onbehalfofrequesterid.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.onbehalfofrequesterid = onbehalfofrequesterid.requesterid)
        OR
        (onbehalfofrequesterid.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND onbehalfofrequesterid.foirequestid = rd.foirequestid AND onbehalfofrequesterid.requesterid = rd.onbehalfofrequesterid)
    LEFT JOIN dimRequesterTypes rqstrt
      ON rr.requestertypeid = rqstrt.requestertypeid
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid
    LEFT JOIN viw_requestinvoices ri
      ON rd.foirequestid = ri.foirequestid

In [0]:
%sql

----------------- Update JOIN dimFOIFlowRequestStatus req to default.viw_avgdaysinrequeststate ----------------
ALTER VIEW default.viw_avgdaysinrequeststate AS
SELECT DISTINCT
  rsd.foiflowrequeststatusdetailsid,
  rsd.runcycleid,
  rsd.foirequestnumber,
  rsd.version,
  rsd.isactive,
  rsd.requeststatusid,
  rsd.ministrycode,
  rsd.assignedgroup,
  rsd.assignedministryperson,
  rsd.assignedministrygroup,
  rsd.startdate,
  rsd.duedate,
  rsd.cfrduedate,
  rsd.closedate,
  rsd.createdby,
  rsd.createddate,
  rsd.modifiedby,
  rsd.modifieddate,
  ffrs.name,
  LAG(ffrs.name) OVER (
      PARTITION BY rsd.foirequestnumber
      ORDER BY rsd.foirequestnumber, rsd.createddate, rsd.requeststatusid
    ) AS laststatusname
FROM
  factFOIFlowRequestStatusDetails rsd
    JOIN dimFOIFlowRequestStatus ffrs
      ON ffrs.requeststatusid = rsd.requeststatusid and ffrs.sourceoftruth = 'FOIMOD'

In [0]:
%sql

----------------- Update default.viw_analystworkloaddash joins to avoid duplicate rows ----------------
ALTER VIEW default.viw_analystworkloaddash AS

  SELECT DISTINCT
  rd.foirequestid,
  rd.visualrequestfilenumber AS requestid,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN 'Intake'
    ELSE ecggn.groupname
  END AS team,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN ecg.groupdescription
    ELSE ecggn.groupdescription
  END AS manager,
  rd.ministry,
  eco.officecode AS procorg,
  rd.applicantcategoryid AS category,
  rt.requesttypename AS type,
  rqt.requestertypename AS applicanttype,
  concat(req.lastname, ', ', req.firstname) AS applicantname,
  frccf.subject,
  rd.startdate,
  rd.closeddate AS enddate,
  rd.targetdate AS duedate,
  rd.processeddays AS totalprocesseddays,
  rd.currentactivity,
  frccf.currentactivitydate,
  rd.primaryusername AS analyst,
  frccf.crossgovtno,
  rd.requestdescription AS description,
  rd.status,
  frccf.customfieldstatus AS `Custom Status`,
  frccf.countontime,
  frccf.countoverdue,
  frccf.onholddays,
  frccf.daysoverdue,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    ELSE 'Not Closed'
  END AS notclosed,
  rpt.estimatedamount AS feesest,
  rpt.incurredamount AS feeswaived,
  rpt.paidamount AS feespaid,
  rd.disposition,
  frccf.publication,
  frccf.publicationreason,
  CASE
    WHEN rd.extension IS NULL THEN 'N'
    ELSE 'Y'
  END AS extension,
  rd.execcomments,
  frccf.noofpagesreleased AS noofpagesdelivered,
  rdd.noofpagesintherequest,
  CASE
    WHEN
      month(rd.receiveddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.receiveddate), year(add_months(rd.receiveddate, 12)))
    WHEN month(rd.receiveddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.receiveddate, -12)), year(rd.receiveddate))
  END AS startfyr,
  CASE
    WHEN
      month(rd.closeddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.closeddate), year(add_months(rd.closeddate, 12)))
    WHEN month(rd.closeddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.closeddate, -12)), year(rd.closeddate))
  END AS endfyr,
  CASE
    WHEN month(rd.receiveddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.receiveddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.receiveddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.receiveddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS startfqtr,
  CASE
    WHEN month(rd.closeddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.closeddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.closeddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.closeddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS endfqtr,
  d.division,
  s.name AS stage,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    WHEN frccf.countontime = 1 THEN 'Active'
    WHEN frccf.countoverdue = 1 THEN 'Overdue'
    ELSE NULL
  END AS overduestatus,
  ffrs.name AS foiflowstatus,
  ffrsd.ministryassigneefullname AS assignedministryperson,
  rds2.divisions,
  rd.requeststatus,
  month(rd.startdate) AS startmonth,
  rd.requesttypeid,
  COALESCE(CAST(ffrsd.recordspagecount AS INT), 0) AS recordspagecount,
  COALESCE(CAST(ffrsd.lanpagecount AS INT), 0) AS lanpagecount
FROM
  factRequestDetails rd
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid and ecg.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimECGroups ecggn
      ON rd.groupname = ecggn.groupname and ecggn.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    LEFT JOIN factRequestPaymentTransaction rpt
      ON rd.foirequestid = rpt.foirequestid
      AND rpt.activeflag = 'Y'
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rd.foirequestid = rdd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes rqt
      ON rd.applicantcategoryid = rqt.requestertypeid and rd.sourceoftruth = rqt.sourceoftruth
    LEFT JOIN dimRequesters req
      ON
        ((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.requesterid = req.requesterid)
        OR
        (req.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid)
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid and rd.sourceoftruth = frccf.sourceoftruth
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factRequestDivisionalStages
      GROUP BY
        foirequestnumber
    ) urds
      ON urds.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestDivisionalStages rds
      ON rds.foirequestnumber = rd.visualrequestfilenumber
      AND urds.runcycleid = rds.runcycleid
      AND urds.createddate = rds.createddate
    LEFT JOIN dimStages s
      ON s.stageid = rds.stageid and s.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimDivisions d
      ON d.divisionid = rds.divisionid and rd.sourceoftruth = d.sourceoftruth
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factFOIFlowRequestStatusDetails
      GROUP BY
        foirequestnumber
    ) uffrsd
      ON uffrsd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factFOIFlowRequestStatusDetails ffrsd
      ON ffrsd.foirequestnumber = rd.visualrequestfilenumber
      AND uffrsd.runcycleid = ffrsd.runcycleid
      AND uffrsd.createddate = ffrsd.createddate
    LEFT JOIN dimFOIFlowRequestStatus ffrs
      ON ffrs.requeststatusid = ffrsd.requeststatusid
    LEFT JOIN (
      SELECT
        rds_1.foirequestnumber,
        string_agg(CONCAT('<p>', d_1.division, ' - ', s_1.name, '</p>'), '\n') AS divisions
      FROM
        (
          SELECT
            foirequestnumber,
            MAX(foirequestversionid) AS foirequestversionid,
            MAX(runcycleid) AS runcycleid
          FROM
            factRequestDivisionalStages
          GROUP BY
            foirequestnumber
        ) sq
          JOIN factRequestDivisionalStages rds_1
            ON rds_1.foirequestnumber = sq.foirequestnumber
            AND rds_1.foirequestversionid = sq.foirequestversionid
            AND rds_1.runcycleid = sq.runcycleid
          JOIN dimStages s_1
            ON s_1.stageid = rds_1.stageid
          JOIN dimDivisions d_1
            ON d_1.divisionid = rds_1.divisionid
      GROUP BY
        rds_1.foirequestnumber
    ) rds2
      ON rds2.foirequestnumber = rd.visualrequestfilenumber
WHERE
  rd.activeflag = 'Y'
  AND rd.officeid NOT IN (1, 10, 21, 33, 41, 45, 49, 51, 55)
  AND rd.requesttypeid IN (31, 33)



In [0]:
%sql

----------------- Update default.viw_analystworkloaddash use officeid for FOIMOD requests ----------------
ALTER VIEW default.viw_analystworkloaddash AS

  SELECT DISTINCT
  rd.foirequestid,
  rd.visualrequestfilenumber AS requestid,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN 'Intake'
    ELSE ecggn.groupname
  END AS team,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN ecg.groupdescription
    ELSE ecggn.groupdescription
  END AS manager,
  rd.ministry,
  CASE
    WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
    ELSE rd.officeid
  END AS procorg,
  rd.applicantcategoryid AS category,
  rt.requesttypename AS type,
  rqt.requestertypename AS applicanttype,
  concat(req.lastname, ', ', req.firstname) AS applicantname,
  frccf.subject,
  rd.startdate,
  rd.closeddate AS enddate,
  rd.targetdate AS duedate,
  rd.processeddays AS totalprocesseddays,
  rd.currentactivity,
  frccf.currentactivitydate,
  rd.primaryusername AS analyst,
  frccf.crossgovtno,
  rd.requestdescription AS description,
  rd.status,
  frccf.customfieldstatus AS `Custom Status`,
  frccf.countontime,
  frccf.countoverdue,
  frccf.onholddays,
  frccf.daysoverdue,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    ELSE 'Not Closed'
  END AS notclosed,
  rpt.estimatedamount AS feesest,
  rpt.incurredamount AS feeswaived,
  rpt.paidamount AS feespaid,
  rd.disposition,
  frccf.publication,
  frccf.publicationreason,
  CASE
    WHEN rd.extension IS NULL THEN 'N'
    ELSE 'Y'
  END AS extension,
  rd.execcomments,
  frccf.noofpagesreleased AS noofpagesdelivered,
  rdd.noofpagesintherequest,
  CASE
    WHEN
      month(rd.receiveddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.receiveddate), year(add_months(rd.receiveddate, 12)))
    WHEN month(rd.receiveddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.receiveddate, -12)), year(rd.receiveddate))
  END AS startfyr,
  CASE
    WHEN
      month(rd.closeddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.closeddate), year(add_months(rd.closeddate, 12)))
    WHEN month(rd.closeddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.closeddate, -12)), year(rd.closeddate))
  END AS endfyr,
  CASE
    WHEN month(rd.receiveddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.receiveddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.receiveddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.receiveddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS startfqtr,
  CASE
    WHEN month(rd.closeddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.closeddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.closeddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.closeddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS endfqtr,
  d.division,
  s.name AS stage,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    WHEN frccf.countontime = 1 THEN 'Active'
    WHEN frccf.countoverdue = 1 THEN 'Overdue'
    ELSE NULL
  END AS overduestatus,
  ffrs.name AS foiflowstatus,
  ffrsd.ministryassigneefullname AS assignedministryperson,
  rds2.divisions,
  rd.requeststatus,
  month(rd.startdate) AS startmonth,
  rd.requesttypeid,
  COALESCE(CAST(ffrsd.recordspagecount AS INT), 0) AS recordspagecount,
  COALESCE(CAST(ffrsd.lanpagecount AS INT), 0) AS lanpagecount
FROM
  factRequestDetails rd
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid and ecg.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimECGroups ecggn
      ON rd.groupname = ecggn.groupname and ecggn.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    LEFT JOIN factRequestPaymentTransaction rpt
      ON rd.foirequestid = rpt.foirequestid
      AND rpt.activeflag = 'Y'
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rd.foirequestid = rdd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes rqt
      ON rd.applicantcategoryid = rqt.requestertypeid and rd.sourceoftruth = rqt.sourceoftruth
    LEFT JOIN dimRequesters req
      ON
        ((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.requesterid = req.requesterid)
        OR
        (req.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid)
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid and rd.sourceoftruth = frccf.sourceoftruth
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factRequestDivisionalStages
      GROUP BY
        foirequestnumber
    ) urds
      ON urds.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestDivisionalStages rds
      ON rds.foirequestnumber = rd.visualrequestfilenumber
      AND urds.runcycleid = rds.runcycleid
      AND urds.createddate = rds.createddate
    LEFT JOIN dimStages s
      ON s.stageid = rds.stageid and s.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimDivisions d
      ON d.divisionid = rds.divisionid and rd.sourceoftruth = d.sourceoftruth
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factFOIFlowRequestStatusDetails
      GROUP BY
        foirequestnumber
    ) uffrsd
      ON uffrsd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factFOIFlowRequestStatusDetails ffrsd
      ON ffrsd.foirequestnumber = rd.visualrequestfilenumber
      AND uffrsd.runcycleid = ffrsd.runcycleid
      AND uffrsd.createddate = ffrsd.createddate
    LEFT JOIN dimFOIFlowRequestStatus ffrs
      ON ffrs.requeststatusid = ffrsd.requeststatusid
    LEFT JOIN (
      SELECT
        rds_1.foirequestnumber,
        string_agg(CONCAT('<p>', d_1.division, ' - ', s_1.name, '</p>'), '\n') AS divisions
      FROM
        (
          SELECT
            foirequestnumber,
            MAX(foirequestversionid) AS foirequestversionid,
            MAX(runcycleid) AS runcycleid
          FROM
            factRequestDivisionalStages
          GROUP BY
            foirequestnumber
        ) sq
          JOIN factRequestDivisionalStages rds_1
            ON rds_1.foirequestnumber = sq.foirequestnumber
            AND rds_1.foirequestversionid = sq.foirequestversionid
            AND rds_1.runcycleid = sq.runcycleid
          JOIN dimStages s_1
            ON s_1.stageid = rds_1.stageid
          JOIN dimDivisions d_1
            ON d_1.divisionid = rds_1.divisionid
      GROUP BY
        rds_1.foirequestnumber
    ) rds2
      ON rds2.foirequestnumber = rd.visualrequestfilenumber
WHERE
  rd.activeflag = 'Y'
  AND rd.officeid NOT IN ("1", "10", "21", "33", "41", "45", "49", "51", "55", 'CLB', 'IIO', 'MGC', 'OBC', 'TIC')
  AND rd.requesttypeid IN (31, 33)



In [0]:
%sql

----------------- use officeid for FOIMOD requests ----------------

ALTER VIEW default.viw_allopenclosed AS
SELECT DISTINCT
  rd.visualrequestfilenumber AS `Request ID`,
  rd.groupname AS `User Group`,
  rd.caseowner AS `Request Owner`,
  rd.ministry AS `Public Body`,
  CASE
    WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
    ELSE rd.officeid
  END AS `Office Code`,
  rt.requesttypename AS `Request Type`,
  category.requestertypename AS `Applicant Category`,
  req.fullname AS `Applicant Name`,
  rd.subject AS `Subject`,
  rd.startdate AS `Start Date`,
  rd.targetdate AS `Due Date`,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN rd.closeddate
    ELSE NULL
  END AS `Closed Date`,
  rd.processeddays AS `Total Process Days`,
  rd.currentactivity AS `Current Action`,
  rd.primaryusername AS `Primary User`,
  rd.crossgovtno AS `Cross Gov #`,
  rd.requestdescription AS `Description`,
  rs.requeststatusname AS `Request Status`,
  custom.countontime AS `Count On Time`,
  custom.countoverdue AS `Count Overdue`,
  custom.onholddays AS `On Hold Days`,
  custom.daysoverdue AS `Days Overdue`,
  pt.estimatedamount AS `Fees Est`,
  pt.feewaived AS `Fees Waived`,
  custom.feepaidamount AS `Fees Paid`,
  rd.disposition AS `Disposition`,
  rd.publication AS `Publication`,
  rd.publicationreason AS `Publication Reason`,
  ext.numberofextensions AS `Extension`,
  rdd.noofpagesintherequest AS `Total # of Pages`,
  frd.recordspagecount AS `Total Mod Pages`
FROM
  factRequestDetails rd
    LEFT JOIN dimRequests r
      ON (
        rd.foirequestid = r.foirequestid
        and rd.sourceoftruth = r.sourceoftruth
      )
      AND r.cactive = 'Y'
    LEFT JOIN dimRequestTypes rt
      ON rt.requesttypeid = rd.requesttypeid
      AND rt.cactive = 'Y'
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rdd.foirequestid = rd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN dimECOffice eco
      ON eco.officeid = rd.officeid
    LEFT JOIN dimRequestStatuses rs
      ON rd.requeststatus = rs.requeststatus
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes category
      ON rd.applicantcategoryid = category.requestertypeid
    LEFT JOIN dimRequesters req
      ON
		((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.requesterid = req.requesterid)
		OR
		(req.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid)
    LEFT JOIN factRequestCustomCalcFields custom
      ON rd.foirequestid = custom.foirequestid
    LEFT JOIN factRequestInvoices ri
      ON rd.foirequestid = ri.foirequestid
      AND ri.activeflag = 'Y'
    LEFT JOIN dimMODMinistryRequestIDs rid
      ON rd.foirequestid = rid.foirequestid
    LEFT JOIN (
      SELECT
        foirequestid,
        COUNT(DISTINCT requestextid) AS numberofextensions
      FROM
        factRequestExtensions
      WHERE
        approvedstatus = 'A'
        AND activeflag = 'Y'
      GROUP BY
        foirequestid
    ) ext
      ON rd.foirequestid = ext.foirequestid
    LEFT JOIN (
      SELECT DISTINCT
        a.foirequestnumber,
        a.recordspagecount
      FROM
        factFOIFlowRequestStatusDetails a
          JOIN (
            SELECT
              foirequestnumber,
              MAX(version) AS maxv
            FROM
              factFOIFlowRequestStatusDetails
            WHERE
              recordspagecount IS NOT NULL
              AND isactive = TRUE
            GROUP BY
              foirequestnumber
          ) c
            ON c.foirequestnumber = a.foirequestnumber
            AND c.maxv = a.version
      WHERE
        a.recordspagecount IS NOT NULL
        AND a.isactive = TRUE
    ) frd
      ON frd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestPaymentTransaction pt
      ON rd.foirequestid = pt.foirequestid
      AND pt.activeflag = 'Y'
    JOIN (
      SELECT
        MAX(runcycleid) AS maxruncycleid,
        visualrequestfilenumber
      FROM
        factRequestDetails
      WHERE
        activeflag = 'Y'
      GROUP BY
        visualrequestfilenumber
    ) maxruncycle
      ON rd.visualrequestfilenumber = maxruncycle.visualrequestfilenumber
      AND rd.runcycleid = maxruncycle.maxruncycleid
WHERE
  rd.activeflag = 'Y'
  AND (
    rd.closeddate >= TIMESTAMP('2019-04-01 00:00:00')
    OR rd.closeddate = TIMESTAMP('1900-01-01 00:00:00')
  )
  AND eco.officecode NOT IN ('IAO', 'IMB', 'XGR', 'PROAC')

In [0]:
%sql

----------------- Use officeid for FOIMOD requests ----------------

ALTER VIEW default.viw_additionalfields AS
SELECT DISTINCT
  rd.visualrequestfilenumber AS `Request ID`,
  frccf.identityverification,
  frccf.previousfoirequest,
  frccf.physicalpageestimate,
  frccf.electronicpageestimate,
  frccf.noofpagesdeduplicated,
  frccf.applicationfeepaid,
  frccf.applicationfeerefunded,
  frccf.applicationfeetransferred,
  frccf.linkedrequests,
  CASE
    WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
    ELSE rd.officeid
  END AS `Action Office`,
  rd.closeddate AS `Closed Date`,
  rd.targetdate AS `Due Date`,
  rd.foirequestid,
  rd.officeid,
  rd.visualrequestfilenumber,
  rd.amendmentcreateddate,
  rd.refvisualrequestfilenumber,
  rd.closedby,
  rd.closeddate,
  rd.notes,
  rd.receivedbyusername,
  rd.createddate,
  rd.currentactivity,
  rd.deliverydate,
  rd.targetdate,
  rd.disposition,
  rd.originalcloseddate,
  rd.originaltargetdate,
  rd.originalreceiveddate,
  rd.primaryusername,
  rd.processeddays,
  rd.receiveddate,
  rd.referencenumber,
  rd.remainingdays,
  rd.requestdescription,
  rd.caseowner,
  rd.status,
  rd.startdate,
  rd.shipaddressid,
  rd.billaddressid,
  rd.receivedmodeid,
  rd.deliverymodeid,
  rd.primarygroupid,
  rd.requesttypeid,
  rd.onbehalfofrequesterid,
  rd.sourceoftruth
FROM
  factRequestDetails rd
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid
WHERE
  rd.activeflag = 'Y'

In [0]:
%sql

----------------- Use officeid for FOIMOD requests ----------------

ALTER VIEW default.viw_sampletlreport AS
SELECT
  rd.visualrequestfilenumber AS Request_ID,
  rt.requesttypename AS Request_Type,
  category.requestertypename AS Applicant_Category,
  rd.redactiondescription AS Request_Description,
  rd.startdate AS Start_Date,
  rd.targetdate AS Due_Date,
  rd.remainingdays AS Remaining_Days,
  rd.currentactivity AS Current_Action,
  rd.requeststatus AS Request_Status,
  rd.primaryusername AS Primary_User,
  rd.execcomments AS Comments,
  rd.extension AS Number_of_Extensions_Applied,
  lastinvoice.invoiceddate AS Last_Invoice_Date,
  rdd.noofpagesintherequest AS Total_pages_in_request,
  eco.officedescription AS Office_Name,
  CASE
    WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
    ELSE rd.officeid
  END AS Office_Code,
  rd.caseowner AS Request_Owner,
  CONCAT(req.lastname, ', ', req.firstname) AS Applicant_Name,
  rd.groupname AS User_Group,
  rd.requesteddate AS Requested_Date,
  rd.receiveddate AS Received_Date,
  rd.closeddate AS Closed_Date,
  rd.originalcloseddate AS Original_Closed_Date,
  rd.requestage AS Request_Age
FROM
  factRequestDetails rd
    LEFT JOIN dimRequests r
      ON (
        rd.foirequestid = r.foirequestid
        and rd.sourceoftruth = r.sourceoftruth
      )
    LEFT JOIN dimRequestTypes rt
      ON rt.requesttypeid = rd.requesttypeid
    LEFT JOIN (
      SELECT
        foirequestid,
        invoiceddate,
        ROW_NUMBER() OVER (PARTITION BY foirequestid ORDER BY createddate DESC) AS rn
      FROM
        factRequestInvoices
      WHERE
        activeflag = 'Y'
    ) lastinvoice
      ON lastinvoice.foirequestid = rd.foirequestid
      AND lastinvoice.rn = 1
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rdd.foirequestid = rd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN dimECOffice eco
      ON eco.officeid = rd.officeid
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes category
      ON rr.requestertypeid = category.requestertypeid
    LEFT JOIN dimRequesters req
      ON
        ((req.sourceoftruth IS NULL OR req.sourceoftruth != 'FOIMOD') AND (rr.sourceoftruth IS NULL OR rr.sourceoftruth != 'FOIMOD') AND rr.requesterid = req.requesterid)
        OR
        (req.sourceoftruth = 'FOIMOD' AND rr.sourceoftruth = 'FOIMOD' AND req.foirequestid = rr.foirequestid AND req.requesterid = rr.requesterid)
WHERE
  rd.activeflag = 'Y'

In [0]:
%sql

-----------------Use officeid for FOIMOD requests ----------------

ALTER VIEW default.viw_broaderaxisfields AS
SELECT
  rd.visualrequestfilenumber AS `Request ID`,
  re.extensiondays AS `# of days extended`,
  frccf.noofdocdelivered AS `# of documents delivered`,
  frccf.noofpagesreleased AS `# of pages released`,
  frccf.noofpagesreviewed AS `# of pages reviewed`,
  CASE
    WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
    ELSE rd.officeid
  END AS `Action Office`,
  rd.amendmentcreateddate AS `Amended Date`,
  rpt.estimatedamount AS `Amount Estimated ($)`,
  rpt.paidamount AS `Amount Paid ($)`,
  concat_ws(
    ', ', abill.address1, abill.address2, abill.city, abill.state, abill.country, abill.zipcode
  ) AS `Applicant Billing Address`,
  rqstrt.requestertypename AS `Applicant Category`,
  rqstr.email AS `Applicant Email`,
  rqstr.fax AS `Applicant Fax`,
  rpt.applicationfee AS `Application Fee ($)`,
  rd.refvisualrequestfilenumber AS `Applicant's File Reference`,
  rqstr.home AS `Applicant Home Phone`,
  rqstr.mobile AS `Applicant Mobile`,
  rqstr.fullname AS `Applicant Name`,
  concat_ws(
    ', ', aship.address1, aship.address2, aship.city, aship.state, aship.country, aship.zipcode
  ) AS `Applicant Shipping Address`,
  rqstr.workphone1 AS `Applicant Work Phone1`,
  rqstr.workphone2 AS `Applicant Work Phone2`,
  rpt.invoicedamount - rpt.paidamount AS `Balance Amount ($)`,
  rd.closedby AS `Closed By`,
  rd.closeddate AS `Closed Date`,
  rd.notes AS `Comments`,
  frccf.coordinatednrresponsereqd AS `Coordinated NR Response Req'd`,
  rd.receivedbyusername AS `Created By`,
  rd.createddate AS `Created Date`,
  rd.currentactivity AS `Current Action`,
  rd.deliverydate AS `Delivery Date`,
  dm.deliverymodename AS `Delivery Mode`,
  rd.targetdate AS `Due Date`,
  re.approveddate AS `Extension Approval/Denial Date`,
  re.approvedcomments AS `Extension Approval/Denial Note`,
  re.completeddate AS `Extension Completed Date`,
  re.completedcomments AS `Extension Completion Note`,
  re.comments AS `Extension Note`,
  et.extensiontypename AS `Extension Reason`,
  CASE
    WHEN
      re.cstatus = 'C'
      AND re.approvedstatus = 'A'
    THEN
      'Completed (Approved)'
    WHEN
      re.cstatus = 'P'
      AND re.approvedstatus = 'N'
    THEN
      'Pending'
    WHEN
      re.cstatus = 'A'
      AND re.approvedstatus = 'A'
    THEN
      'Approved'
    WHEN
      re.cstatus = 'D'
      AND re.approvedstatus = 'D'
    THEN
      'Denied'
    WHEN
      re.cstatus = 'C'
      AND re.approvedstatus = 'D'
    THEN
      'Completed (Denied)'
    ELSE NULL
  END AS `Extension Status`,
  rpt.feewaivedescription AS `Fee Waiver Request Reason`,
  rpt.feewaiverequested AS `Fee Waiver Requested`,
  rpt.feewaivegranted AS `Fee Waiver Status`,
  rd.disposition AS `Final Disposition`,
  rpt.invoicedamount AS `Last Invoice Amount ($)`,
  rpt.invoicedate AS `Last Invoice Date`,
  rpt.invoicenumber AS `Last Invoice Number`,
  frccf.applicantfilereference AS `Link Requests`,
  pm.paymentmodename AS `Method of Payment`,
  frccf.noofpagesdeduplicated AS `Number of Pages Deduplicated`,
  onbehalfofrequesterid.fullname AS `On Behalf Of`,
  rqstr.company AS `Organization`,
  rd.originalcloseddate AS `Original Closed Date`,
  rd.originaltargetdate AS `Original Due Date`,
  rd.originalreceiveddate AS `Original Start Date`,
  rpt.paymentreceiveddate AS `Payment Date`,
  rpt.paymentstatus AS `Payment Status`,
  rd.primaryusername AS `Primary User`,
  ecg.groupname AS `Primary User Group`,
  rd.processeddays AS `Processed Days`,
  rd.receiveddate AS `Received Date`,
  rm.receivedmodename AS `Received Mode`,
  rd.referencenumber AS `Reference Number`,
  frccf.refundamount AS `Refund Amount`,
  rd.remainingdays AS `Remaining Days`,
  rd.requestdescription AS `Request Description`,
  rd.caseowner AS `Request Owner`,
  rd.status AS `Request Status`,
  rt.requesttypename AS `Request Type`,
  frccf.secondaryusers AS `Secondary Users`,
  rd.startdate AS `Start Date`,
  frccf.subject AS `Subject`,
  rdd.noofpagesintherequest AS `Total # of pages in a request`,
  rdd.noofpagesinreviewlog AS `Total # of pages in the review log`,
  frccf.onholddays AS `Total Days on Hold`,
  rpt.totalinvoicedamount AS `Total Invoiced ($)`,
  frccf.crossgovtno AS `XGOV`,
  COALESCE(ri.chargedamount, 0) AS `Charged Amount ($)`
FROM
  viw_additionalfields rd
    LEFT JOIN dimRequests r
      ON rd.foirequestid = r.foirequestid
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN dimAddress aship
      ON
        ((aship.sourceoftruth IS NULL OR aship.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.shipaddressid = aship.addressid)
        OR
        (aship.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND aship.foirequestid = rd.foirequestid)
    LEFT JOIN dimAddress abill
      ON
        ((abill.sourceoftruth IS NULL OR abill.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.shipaddressid = abill.addressid)
        OR
        (abill.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND abill.foirequestid = rd.foirequestid)
    LEFT JOIN factRequestExtensions re
      ON rd.foirequestid = re.foirequestid
      AND re.activeflag = 'Y'
    LEFT JOIN dimExtensionTypes et
      ON re.extensiontypeid = et.extensiontypeid
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesters rqstr
      ON
        ((rqstr.sourceoftruth IS NULL OR rqstr.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rr.requesterid = rqstr.requesterid)
        OR
        (rqstr.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND rqstr.foirequestid = rd.foirequestid AND rqstr.requesterid = rr.requesterid)
    LEFT JOIN dimReceivedModes rm
      ON rd.receivedmodeid = rm.receivedmodeid
    LEFT JOIN dimDeliveryModes dm
      ON rd.deliverymodeid = dm.deliverymodeid
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rd.foirequestid = rdd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN factRequestPaymentTransaction rpt
      ON rd.foirequestid = rpt.foirequestid
      AND rpt.activeflag = 'Y'
    LEFT JOIN dimPaymentModes pm
      ON rpt.paymentmodeid = pm.paymentmodeid
    LEFT JOIN dimRequesters onbehalfofrequesterid
      ON
        ((onbehalfofrequesterid.sourceoftruth IS NULL OR onbehalfofrequesterid.sourceoftruth != 'FOIMOD') AND (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') AND rd.onbehalfofrequesterid = onbehalfofrequesterid.requesterid)
        OR
        (onbehalfofrequesterid.sourceoftruth = 'FOIMOD' AND rd.sourceoftruth = 'FOIMOD' AND onbehalfofrequesterid.foirequestid = rd.foirequestid AND onbehalfofrequesterid.requesterid = rd.onbehalfofrequesterid)
    LEFT JOIN dimRequesterTypes rqstrt
      ON rr.requestertypeid = rqstrt.requestertypeid
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid
    LEFT JOIN viw_requestinvoices ri
      ON rd.foirequestid = ri.foirequestid

In [0]:
%sql
----------------- Update default.viw_analystworkloaddash fix error from coalescing null values to int ----------------
ALTER VIEW default.viw_analystworkloaddash AS

SELECT DISTINCT
  rd.foirequestid,
  rd.visualrequestfilenumber AS requestid,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN 'Intake'
    ELSE ecggn.groupname
  END AS team,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN ecg.groupdescription
    ELSE ecggn.groupdescription
  END AS manager,
  rd.ministry,
  CASE
    WHEN
      (
        rd.sourceoftruth IS NULL
        OR rd.sourceoftruth != 'FOIMOD'
      )
    THEN
      eco.officecode
    ELSE rd.officeid
  END AS procorg,
  rd.applicantcategoryid AS category,
  rt.requesttypename AS type,
  rqt.requestertypename AS applicanttype,
  concat(req.lastname, ', ', req.firstname) AS applicantname,
  frccf.subject,
  rd.startdate,
  rd.closeddate AS enddate,
  rd.targetdate AS duedate,
  rd.processeddays AS totalprocesseddays,
  rd.currentactivity,
  frccf.currentactivitydate,
  rd.primaryusername AS analyst,
  frccf.crossgovtno,
  rd.requestdescription AS description,
  rd.status,
  frccf.customfieldstatus AS `Custom Status`,
  frccf.countontime,
  frccf.countoverdue,
  frccf.onholddays,
  frccf.daysoverdue,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    ELSE 'Not Closed'
  END AS notclosed,
  rpt.estimatedamount AS feesest,
  rpt.incurredamount AS feeswaived,
  rpt.paidamount AS feespaid,
  rd.disposition,
  frccf.publication,
  frccf.publicationreason,
  CASE
    WHEN rd.extension IS NULL THEN 'N'
    ELSE 'Y'
  END AS extension,
  rd.execcomments,
  frccf.noofpagesreleased AS noofpagesdelivered,
  rdd.noofpagesintherequest,
  CASE
    WHEN
      month(rd.receiveddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.receiveddate), year(add_months(rd.receiveddate, 12)))
    WHEN month(rd.receiveddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.receiveddate, -12)), year(rd.receiveddate))
  END AS startfyr,
  CASE
    WHEN
      month(rd.closeddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.closeddate), year(add_months(rd.closeddate, 12)))
    WHEN month(rd.closeddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.closeddate, -12)), year(rd.closeddate))
  END AS endfyr,
  CASE
    WHEN month(rd.receiveddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.receiveddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.receiveddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.receiveddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS startfqtr,
  CASE
    WHEN month(rd.closeddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.closeddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.closeddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.closeddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS endfqtr,
  d.division,
  s.name AS stage,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    WHEN frccf.countontime = 1 THEN 'Active'
    WHEN frccf.countoverdue = 1 THEN 'Overdue'
    ELSE NULL
  END AS overduestatus,
  ffrs.name AS foiflowstatus,
  ffrsd.ministryassigneefullname AS assignedministryperson,
  rds2.divisions,
  rd.requeststatus,
  month(rd.startdate) AS startmonth,
  rd.requesttypeid,
  COALESCE(TRY_CAST(ffrsd.recordspagecount AS INT), 0) AS recordspagecount,
  COALESCE(TRY_CAST(ffrsd.lanpagecount AS INT), 0) AS lanpagecount
FROM
  factRequestDetails rd
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid
      and ecg.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimECGroups ecggn
      ON rd.groupname = ecggn.groupname
      and ecggn.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    LEFT JOIN factRequestPaymentTransaction rpt
      ON rd.foirequestid = rpt.foirequestid
      AND rpt.activeflag = 'Y'
    LEFT JOIN factRequestDocumentsDetails rdd
      ON rd.foirequestid = rdd.foirequestid
      AND rdd.activeflag = 'Y'
    LEFT JOIN factRequestRequesters rr
      ON rd.foirequestid = rr.foirequestid
      AND rr.activeflag = 'Y'
    LEFT JOIN dimRequesterTypes rqt
      ON rd.applicantcategoryid = rqt.requestertypeid
      and rd.sourceoftruth = rqt.sourceoftruth
    LEFT JOIN dimRequesters req
      ON (
        (
          req.sourceoftruth IS NULL
          OR req.sourceoftruth != 'FOIMOD'
        )
        AND (
          rd.sourceoftruth IS NULL
          OR rd.sourceoftruth != 'FOIMOD'
        )
        AND rd.requesterid = req.requesterid
      )
      OR (
        req.sourceoftruth = 'FOIMOD'
        AND rd.sourceoftruth = 'FOIMOD'
        AND req.foirequestid = rd.foirequestid
        AND req.requesterid = rd.requesterid
      )
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid
      and rd.sourceoftruth = frccf.sourceoftruth
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factRequestDivisionalStages
      GROUP BY
        foirequestnumber
    ) urds
      ON urds.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestDivisionalStages rds
      ON rds.foirequestnumber = rd.visualrequestfilenumber
      AND urds.runcycleid = rds.runcycleid
      AND urds.createddate = rds.createddate
    LEFT JOIN dimStages s
      ON s.stageid = rds.stageid
      and s.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimDivisions d
      ON d.divisionid = rds.divisionid
      and rd.sourceoftruth = d.sourceoftruth
    LEFT JOIN (
      SELECT
        foirequestnumber,
        MAX(runcycleid) AS runcycleid,
        MAX(createddate) AS createddate
      FROM
        factFOIFlowRequestStatusDetails
      GROUP BY
        foirequestnumber
    ) uffrsd
      ON uffrsd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factFOIFlowRequestStatusDetails ffrsd
      ON ffrsd.foirequestnumber = rd.visualrequestfilenumber
      AND uffrsd.runcycleid = ffrsd.runcycleid
      AND uffrsd.createddate = ffrsd.createddate
    LEFT JOIN dimFOIFlowRequestStatus ffrs
      ON ffrs.requeststatusid = ffrsd.requeststatusid
    LEFT JOIN (
      SELECT
        rds_1.foirequestnumber,
        string_agg(CONCAT('<p>', d_1.division, ' - ', s_1.name, '</p>'), '\n') AS divisions
      FROM
        (
          SELECT
            foirequestnumber,
            MAX(foirequestversionid) AS foirequestversionid,
            MAX(runcycleid) AS runcycleid
          FROM
            factRequestDivisionalStages
          GROUP BY
            foirequestnumber
        ) sq
          JOIN factRequestDivisionalStages rds_1
            ON rds_1.foirequestnumber = sq.foirequestnumber
            AND rds_1.foirequestversionid = sq.foirequestversionid
            AND rds_1.runcycleid = sq.runcycleid
          JOIN dimStages s_1
            ON s_1.stageid = rds_1.stageid
          JOIN dimDivisions d_1
            ON d_1.divisionid = rds_1.divisionid
      GROUP BY
        rds_1.foirequestnumber
    ) rds2
      ON rds2.foirequestnumber = rd.visualrequestfilenumber
WHERE
  rd.activeflag = 'Y'
  AND rd.officeid NOT IN (
    "1", "10", "21", "33", "41", "45", "49", "51", "55", 'CLB', 'IIO', 'MGC', 'OBC', 'TIC'
  )
  AND rd.requesttypeid IN (31, 33)

In [0]:
%sql

-- get data from materialized view
ALTER VIEW default.viw_allopenclosed AS
SELECT
    visualrequestfilenumber AS `Request ID`,
    groupname AS `User Group`,
    caseowner AS `Request Owner`,
    ministry AS `Public Body`,
    officecode AS `Office Code`,
    requesttypename AS `Request Type`,
    requestertypename AS `Applicant Category`,
    fullname AS `Applicant Name`,
    subject AS `Subject`,
    startdate AS `Start Date`,
    try_cast(targetdate AS DATE) AS `Due Date`,
    closeddate AS `Closed Date`,
    processeddays AS `Total Process Days`,
    currentactivity AS `Current Action`,
    primaryusername AS `Primary User`,
    crossgovtno AS `Cross Gov #`,
    requestdescription AS `Description`,
    requeststatusname AS `Request Status`,
    countontime AS `Count On Time`,
    countoverdue AS `Count Overdue`,
    onholddays AS `On Hold Days`,
    daysoverdue AS `Days Overdue`,
    estimatedamount AS `Fees Est`,
    feewaived AS `Fees Waived`,
    feepaidamount AS `Fees Paid`,
    disposition AS `Disposition`,
    publication AS `Publication`,
    publicationreason AS `Publication Reason`,
    numberofextensions AS `Extension`,
    noofpagesintherequest AS `Total # of Pages`,
    try_cast(recordspagecount AS BIGINT) AS `Total Mod Pages`
FROM default.materialized_viw_allopenclosed;

In [0]:
%sql

-- get data from materialized views
ALTER VIEW default.viw_analystworkloaddash AS

  SELECT
  foirequestid,
  requestid,
  team,
  manager,
  ministry,
  procorg,
  category,
  type,
  applicanttype,
  applicantname,
  subject,
  startdate,
  enddate,
  duedate,
  totalprocesseddays,
  currentactivity,
  currentactivitydate,
  analyst,
  crossgovtno,
  description,
  status,
  CustomStatus AS `Custom Status`,
  countontime,
  countoverdue,
  onholddays,
  daysoverdue,
  notclosed,
  feesest,
  feeswaived,
  feespaid,
  disposition,
  publication,
  publicationreason,
  extension,
  execcomments,
  noofpagesdelivered,
  noofpagesintherequest,
  startfyr,
  endfyr,
  startfqtr,
  endfqtr,
  division,
  stage,
  overduestatus,
  foiflowstatus,
  assignedministryperson,
  divisions,
  requeststatus,
  startmonth,
  requesttypeid,
  recordspagecount,
  lanpagecount
FROM
  default.materialized_viw_analystworkloaddash;



In [0]:
%sql

----------------- try_cast to bigint to resolve cast_invalid_input issue ----------------

ALTER VIEW default.viw_additionalfields AS
SELECT DISTINCT
  rd.visualrequestfilenumber AS `Request ID`,
  frccf.identityverification,
  frccf.previousfoirequest,
  try_cast(frccf.physicalpageestimate AS INT) AS physicalpageestimate,
  try_cast(frccf.electronicpageestimate AS INT) AS electronicpageestimate,
  try_cast(frccf.noofpagesdeduplicated AS INT) AS noofpagesdeduplicated,
  frccf.applicationfeepaid,
  frccf.applicationfeerefunded,
  frccf.applicationfeetransferred,
  frccf.linkedrequests,
  CASE
    WHEN
      (
        rd.sourceoftruth IS NULL
        OR rd.sourceoftruth != 'FOIMOD'
      )
    THEN
      eco.officecode
    ELSE rd.officeid
  END AS `Action Office`,
  rd.closeddate AS `Closed Date`,
  rd.targetdate AS `Due Date`,
  try_cast(rd.foirequestid AS BIGINT) AS foirequestid,
  rd.officeid,
  rd.visualrequestfilenumber,
  rd.amendmentcreateddate,
  rd.refvisualrequestfilenumber,
  rd.closedby,
  rd.closeddate,
  rd.notes,
  rd.receivedbyusername,
  rd.createddate,
  rd.currentactivity,
  rd.deliverydate,
  rd.targetdate,
  rd.disposition,
  rd.originalcloseddate,
  rd.originaltargetdate,
  rd.originalreceiveddate,
  rd.primaryusername,
  try_cast(rd.processeddays AS BIGINT) AS processeddays,
  rd.receiveddate,
  rd.referencenumber,
  try_cast(rd.remainingdays AS BIGINT) AS remainingdays,
  rd.requestdescription,
  rd.caseowner,
  rd.status,
  rd.startdate,
  try_cast(rd.shipaddressid AS BIGINT) AS shipaddressid,
  try_cast(rd.billaddressid AS BIGINT) AS billaddressid,
  rd.receivedmodeid,
  rd.deliverymodeid,
  try_cast(rd.primarygroupid AS BIGINT) AS primarygroupid,
  try_cast(rd.requesttypeid AS BIGINT) AS requesttypeid,
  rd.onbehalfofrequesterid,
  rd.sourceoftruth
FROM
  factRequestDetails rd
    LEFT JOIN dimECOffice eco
      ON rd.officeid = eco.officeid
    LEFT JOIN factRequestCustomCalcFields frccf
      ON rd.foirequestid = frccf.foirequestid
WHERE
  rd.activeflag = 'Y'

In [0]:
%sql

-- get data from materialized views for viw_additionalfields now
ALTER VIEW default.viw_additionalfields AS
  SELECT
  requestid AS `Request ID`,
  identityverification,
  previousfoirequest,
  physicalpageestimate,
  electronicpageestimate,
  noofpagesdeduplicated,
  applicationfeepaid,
  applicationfeerefunded,
  applicationfeetransferred,
  linkedrequests,
  actionoffice AS `Action Office`,
  closedate AS `Closed Date`,
  duedate AS `Due Date`,
  foirequestid,
  officeid,
  visualrequestfilenumber,
  amendmentcreateddate,
  refvisualrequestfilenumber,
  closedby,
  closeddate,
  notes,
  receivedbyusername,
  createddate,
  currentactivity,
  deliverydate,
  targetdate,
  disposition,
  originalcloseddate,
  originaltargetdate,
  originalreceiveddate,
  primaryusername,
  processeddays,
  receiveddate,
  referencenumber,
  remainingdays,
  requestdescription,
  caseowner,
  status,
  startdate,
  shipaddressid,
  billaddressid,
  receivedmodeid,
  deliverymodeid,
  primarygroupid,
  requesttypeid,
  onbehalfofrequesterid,
  sourceoftruth
FROM
  default.materialized_viw_additionalfields;

In [0]:
%sql

-- get data from materialized views for viw_activeoipcreviews now
ALTER VIEW default.viw_activeoipcreviews AS
  SELECT
  visualrequestfilenumber AS `Request ID`,
  oipcno AS `OIPC No`,
  primaryusername AS `Primary User`,
  startdate AS `Start Date`,
  targetdate AS `Due Date`,
  reviewtype AS `Review Type`,
  customfieldstatus AS `Status`,
  reason AS `Reason`,
  requestdescription AS `Request Description`,
  subject AS `Subject`,
  requestertypename AS `Applicant Type`,
  portfolioofficer AS `Portfolio Officer`,
  judicialreview AS `Judicial Review`,
  status AS `Request Status`,
  requeststatusname AS `Request Status Name`,
  procorg
FROM
  default.materialized_viw_activeoipcreviews;

In [0]:
%sql

-- get data from materialized views for viw_broaderaxisfields now
ALTER VIEW default.viw_broaderaxisfields AS
  SELECT
  visualrequestfilenumber AS `Request ID`,
  extensiondays AS `# of days extended`,
  noofdocdelivered AS `# of documents delivered`,
  noofpagesreleased AS `# of pages released`,
  noofpagesreviewed AS `# of pages reviewed`,
  actionoffice AS `Action Office`,
  amendmentcreateddate AS `Amended Date`,
  estimatedamount AS `Amount Estimated ($)`,
  paidamount AS `Amount Paid ($)`,
  applicantbillingaddress AS `Applicant Billing Address`,
  requestertypename AS `Applicant Category`,
  email AS `Applicant Email`,
  fax AS `Applicant Fax`,
  applicationfee AS `Application Fee ($)`,
  refvisualrequestfilenumber AS `Applicant's File Reference`,
  home AS `Applicant Home Phone`,
  mobile AS `Applicant Mobile`,
  fullname AS `Applicant Name`,
  applicantshippingaddress AS `Applicant Shipping Address`,
  workphone1 AS `Applicant Work Phone1`,
  workphone2 AS `Applicant Work Phone2`,
  balanceamount AS `Balance Amount ($)`,
  closedby AS `Closed By`,
  closeddate AS `Closed Date`,
  notes AS `Comments`,
  coordinatednrresponsereqd AS `Coordinated NR Response Req'd`,
  receivedbyusername AS `Created By`,
  createddate AS `Created Date`,
  currentactivity AS `Current Action`,
  deliverydate AS `Delivery Date`,
  deliverymodename AS `Delivery Mode`,
  targetdate AS `Due Date`,
  approveddate AS `Extension Approval/Denial Date`,
  approvedcomments AS `Extension Approval/Denial Note`,
  completeddate AS `Extension Completed Date`,
  completedcomments AS `Extension Completion Note`,
  comments AS `Extension Note`,
  extensiontypename AS `Extension Reason`,
  extensionstatus AS `Extension Status`,
  feewaivedescription AS `Fee Waiver Request Reason`,
  feewaiverequested AS `Fee Waiver Requested`,
  feewaivegranted AS `Fee Waiver Status`,
  disposition AS `Final Disposition`,
  invoicedamount AS `Last Invoice Amount ($)`,
  invoicedate AS `Last Invoice Date`,
  invoicenumber AS `Last Invoice Number`,
  applicantfilereference AS `Link Requests`,
  paymentmodename AS `Method of Payment`,
  noofpagesdeduplicated AS `Number of Pages Deduplicated`,
  onbehalfof AS `On Behalf Of`,
  company AS `Organization`,
  originalcloseddate AS `Original Closed Date`,
  originaltargetdate AS `Original Due Date`,
  originalreceiveddate AS `Original Start Date`,
  paymentreceiveddate AS `Payment Date`,
  paymentstatus AS `Payment Status`,
  primaryusername AS `Primary User`,
  groupname AS `Primary User Group`,
  processeddays AS `Processed Days`,
  receiveddate AS `Received Date`,
  receivedmodename AS `Received Mode`,
  referencenumber AS `Reference Number`,
  refundamount AS `Refund Amount`,
  remainingdays AS `Remaining Days`,
  requestdescription AS `Request Description`,
  caseowner AS `Request Owner`,
  status AS `Request Status`,
  requesttypename AS `Request Type`,
  secondaryusers AS `Secondary Users`,
  startdate AS `Start Date`,
  subject AS `Subject`,
  noofpagesintherequest AS `Total # of pages in a request`,
  noofpagesinreviewlog AS `Total # of pages in the review log`,
  onholddays AS `Total Days on Hold`,
  totalinvoicedamount AS `Total Invoiced ($)`,
  crossgovtno AS `XGOV`,
  chargedamount AS `Charged Amount ($)`
FROM
  default.materialized_viw_broaderaxisfields;

In [0]:
%sql

-- get data from materialized views for viw_requestinvoices now
ALTER VIEW default.viw_requestinvoices AS
  SELECT
  chargedamount,
  foirequestid,
  sourceoftruth
FROM
  default.materialized_viw_requestinvoices;

In [0]:
%sql

-- get data from materialized views for viw_cfrrfdreport now
ALTER VIEW default.viw_cfrrfdreport AS
  SELECT
    *
  FROM
    default.materialized_viw_cfrrfdreport;

In [0]:
%sql

-- get data from materialized views for viw_avgdaysinrequeststate now
ALTER VIEW default.viw_avgdaysinrequeststate AS
SELECT *
FROM
  default.materialized_viw_avgdaysinrequeststate

In [0]:
%sql

-- get data from materialized views for viw_oipc_10_1dreport now
ALTER VIEW default.viw_oipc_10_1dreport AS
SELECT *
FROM
  default.materialized_viw_oipc_10_1dreport

In [0]:
%sql

-- get data from materialized views for viw_scanningreport now
ALTER VIEW default.viw_scanningreport AS
SELECT *
FROM
  default.materialized_viw_scanningreport

In [0]:
%sql

-- get data from materialized views for viw_scanningreport now
CREATE OR REPLACE VIEW default.viw_eccrequeststate AS
SELECT *
FROM
  default.materialized_viw_eccrequeststate

In [0]:
%sql

-- create view for performance dashboard
CREATE OR REPLACE VIEW default.viw_performance_ontimeoverdue AS
  Select 
      'On Time' as Category,
      sum(countontime) as `Count`
      -- sum(countoverdue) as "Overdue" 
  from default.viw_analystworkloaddash
  where notclosed = 'Not Closed'
  union all
  Select 
      'Overdue' as Category,
      sum(countoverdue) as `Count` 
  from default.viw_analystworkloaddash
  Where --procorg not in ('CLB', 'HSA', 'IIO', 'MGC', 'OBC') and
  notclosed = 'Not Closed'

In [0]:
%sql

-- create view for performance dashboard
CREATE OR REPLACE VIEW default.viw_performance_dispositions AS
  SELECT 
      disposition as `Disposition`,
      count(disposition) as `DispositionCount`
  from default.viw_analystworkloaddash 
  Where --procorg not in ('CLB', 'HSA', 'IIO', 'MGC', 'OBC') and 
  requeststatus = 'Closed' 
  group by disposition

In [0]:
%sql

-- create view for performance dashboard
CREATE OR REPLACE VIEW default.viw_performance_orgavgdaysbystate AS
  SELECT 
    name as state, 
    ministrycode, 
    AVG(DATEDIFF(statusenddate, createddate)) AS avg_days
  FROM (
    SELECT *, 
      LEAD(createddate) OVER (
        PARTITION BY foirequestnumber 
        ORDER BY createddate, foirequestnumber, name
      ) AS statusenddate
    FROM (
      SELECT *
      FROM default.viw_avgdaysinrequeststate
      WHERE name != laststatusname OR laststatusname IS NULL
    ) sssq
  ) ssq
  WHERE statusenddate IS NOT NULL
    AND name IN (
      'Call For Records', 'Open', 'Consult', 'Deduplication', 'Fee Estimate',
      'Harms Assessment', 'Ministry Sign Off', 'On Hold', 'Peer Review',
      'Records Review', 'Response', 'Closed', 'Intake in Progress',
      'Section 5 Pending', 'App Fee Owing'
    )
    AND ministrycode NOT IN ('HLTH')
  GROUP BY ministrycode, name
  ORDER BY avg_days DESC

In [0]:
%sql

-- create view for analyst workload dashboard
CREATE OR REPLACE VIEW default.viw_analystworkloaddashboard AS
select requestid, type, currentactivity, publication, team, procorg, analyst, duedate, status FROM viw_analystworkloaddash
    where procorg not in ('CLB', 'HSA', 'IIO', 'MGC', 'OBC')
and status != 'Closed'

In [0]:
%sql

-- create view for analyst workload dashboard
CREATE OR REPLACE VIEW default.viw_analystworkloaddashboard_details AS
SELECT 
  requestid AS `Request #`,
  DATE(duedate) AS `Due Date`,
  vw.status AS `Status`,
  currentactivity AS `Current Activity`,
  CASE 
    WHEN procorg IN ('CFD', 'JAG', 'MAG', 'MSD', 'PSS') THEN subject 
    ELSE description 
  END AS `Description`,
  analyst AS `Analyst`,
  team AS `Team`,
  applicanttype AS `Applicant Type`,
  noofpagesintherequest AS `AXIS Page Count`,
  recordspagecount AS `FOI Mod Page Count`,
  publication AS `Publication`,
  feesest AS `Fees Est`,
  feeswaived AS `Fees Waived`,
  feespaid AS `Fees Paid`,
  subject AS `Subject`,
  totalprocesseddays AS `Total Process Days`,
  daysoverdue AS `Days Overdue`,
  team AS `Team/Sector`,
  extension AS `Extensions`
FROM default.viw_analystworkloaddash vw
WHERE 
  procorg NOT IN ('CLB', 'HSA', 'IIO', 'MGC', 'OBC')
  AND status != 'Closed'

In [0]:
%sql

-- create view for active oipc reviews report
CREATE OR REPLACE VIEW default.viw_activeoipcreviewsreport AS
SELECT 
  `Request ID`, 
  `OIPC No` AS `OIPC No`, 
  `Primary User`,
  `Start Date`,
  `Due Date`,
  `Review Type` AS `Review Type`,
  `Status` AS `Status`,
  `Reason` AS `Reason`,
  CASE 
    WHEN procorg IN ('CFD', 'JAG', 'MAG', 'MSD', 'PSS') 
      THEN `Subject` 
      ELSE `Request Description` 
  END AS `Request Description`,
  `Subject`,
  `Applicant Type`,
  `Portfolio Officer` AS `Portfolio Officer`,
  `Judicial Review` AS `Judicial Review`
FROM default.viw_activeoipcreviews

In [0]:
%sql

-- create view for unopened report
CREATE OR REPLACE VIEW default.viw_unopenedreport AS
SELECT 
  ur.date AS `Report Date`,
  CONCAT('U-000', CAST(ur.rawrequestid AS STRING)) AS `Request ID`,
  ur.rank AS `Rank`,
  try_cast(get_json_object(rr.requestrawdata, '$.receivedDateUF') AS DATE) AS `Received Date`,
  STRING_AGG(code.code, ', ') AS `Ministries`,
  get_json_object(rr.requestrawdata, '$.contactInfo.firstName') AS `First Name`,
  get_json_object(rr.requestrawdata, '$.contactInfo.lastName') AS `Last Name`,
  COALESCE(p.status, '') AS `Payment status`,
  COALESCE(p.transaction_number, '') AS `Receipt No.`,
  COALESCE(CAST(p.total AS STRING), '') AS `Total Amount`,
  REPLACE(
    REPLACE(
      REPLACE(
        REPLACE(
          REPLACE(
            get_json_object(ur.potentialmatches, '$.matches'),
            '}, {"requestid": "', '<br>'
          ),
          '", "', ' - '
        ),
        '[{"requestid": "', ''
      ),
      '}]', ''
    ),
    'similarity"', 'similarity'
  ) AS `Potential Matches`,
  get_json_object(rr.requestrawdata, '$.descriptionTimeframe.description') AS `Description`
FROM foi_mod.UnopenedReport ur
JOIN foi_mod.FOIRawRequests rr ON rr.requestid = ur.rawrequestid AND rr.version = 1
LEFT JOIN (
    SELECT request_id, MAX(payment_id) AS max_payment_id
    FROM foi_mod.Payments
    WHERE fee_code_id = 1
    GROUP BY request_id
) mp ON mp.request_id = rr.requestid
LEFT JOIN foi_mod.Payments p 
  ON p.payment_id = mp.max_payment_id
LATERAL VIEW EXPLODE(
  FROM_JSON(
    get_json_object(rr.requestrawdata, '$.ministry.selectedMinistry'),
    'array<struct<code:string>>'
  )
) AS code
GROUP BY 
  ur.date,
  CONCAT('U-000', CAST(ur.rawrequestid AS STRING)),
  ur.rank,
  try_cast(get_json_object(rr.requestrawdata, '$.receivedDateUF') AS DATE),
  get_json_object(rr.requestrawdata, '$.contactInfo.firstName'),
  get_json_object(rr.requestrawdata, '$.contactInfo.lastName'),
  COALESCE(p.status, ''),
  COALESCE(p.transaction_number, ''),
  COALESCE(CAST(p.total AS STRING), ''),
  get_json_object(ur.potentialmatches, '$.matches'),
  get_json_object(rr.requestrawdata, '$.descriptionTimeframe.description'),
  ur.id,
  rr.created_at
ORDER BY rr.created_at

In [0]:
%sql

-- create init allopenclosed materialized view based on AXIS DATA

CREATE OR REPLACE TABLE default.materialized_viw_allopenclosed AS
SELECT DISTINCT
    rd.visualrequestfilenumber,
    rd.groupname,
    rd.caseowner,
    rd.ministry,
    eco.officecode,
    rt.requesttypename,
    category.requestertypename,
    req.fullname,
    rd.subject,
    rd.startdate,
    rd.targetdate,
    CASE
        WHEN rd.requeststatus = 'Closed' THEN rd.closeddate
        ELSE NULL
    END AS closeddate,
    rd.processeddays,
    rd.currentactivity,
    rd.primaryusername,
    rd.crossgovtno,
    rd.requestdescription,
    rs.requeststatusname,
    custom.countontime,
    custom.countoverdue,
    custom.onholddays,
    custom.daysoverdue,
    pt.estimatedamount,
    pt.feewaived,
    custom.feepaidamount,
    rd.disposition,
    rd.publication,
    rd.publicationreason,
    ext.numberofextensions,
    rdd.noofpagesintherequest,
    try_cast(frd.recordspagecount AS BIGINT) AS recordspagecount
FROM
    (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestDetails
      WHERE
          sourceoftruth = 'AXIS' AND activeflag = 'Y'
    ) AS rd
    -- factRequestDetails AS rd
    LEFT JOIN dimRequests AS r ON (rd.foirequestid = r.foirequestid AND rd.sourceoftruth = r.sourceoftruth) AND r.cactive = 'Y'
    LEFT JOIN dimRequestTypes AS rt ON rt.requesttypeid = rd.requesttypeid AND rt.cactive = 'Y'
    -- LEFT JOIN factRequestDocumentsDetails AS rdd ON rdd.foirequestid = rd.foirequestid
    -- AND rdd.activeflag = 'Y'
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestDocumentsDetails
        WHERE
            sourceoftruth = 'AXIS' AND activeflag = 'Y'
    ) AS rdd ON rd.foirequestid = rdd.foirequestid AND rdd.rn = 1
    LEFT JOIN dimECOffice AS eco ON eco.officeid = rd.officeid
    LEFT JOIN dimRequestStatuses AS rs ON rd.requeststatus = rs.requeststatus
    -- LEFT JOIN factRequestRequesters AS rr ON rd.foirequestid = rr.foirequestid AND rr.activeflag = 'Y'
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestRequesters
        WHERE
            sourceoftruth = 'AXIS' AND activeflag = 'Y'
    ) AS rr ON rd.foirequestid = rr.foirequestid AND rr.rn = 1
    LEFT JOIN dimRequesterTypes AS category ON rd.applicantcategoryid = category.requestertypeid AND rd.sourceoftruth = category.sourceoftruth
    LEFT JOIN dimRequesters AS req ON req.sourceoftruth = rd.sourceoftruth AND rd.requesterid = req.requesterid
    -- LEFT JOIN factRequestCustomCalcFields AS custom ON rd.foirequestid = custom.foirequestid
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestCustomCalcFields
        WHERE
            sourceoftruth = 'AXIS'
    ) AS custom ON rd.foirequestid = custom.foirequestid AND custom.rn = 1
    LEFT JOIN factRequestInvoices AS ri ON rd.foirequestid = ri.foirequestid
    AND ri.activeflag = 'Y'
    LEFT JOIN dimMODMinistryRequestIDs AS rid ON rd.foirequestid = rid.foirequestid
    LEFT JOIN (
        SELECT
            foirequestid,
            COUNT(DISTINCT requestextid) AS numberofextensions
        FROM
            factRequestExtensions
        WHERE
            sourceoftruth = 'AXIS'
            AND approvedstatus = 'A'
            AND activeflag = 'Y'
        GROUP BY
            foirequestid
    ) AS ext ON rd.foirequestid = ext.foirequestid
    LEFT JOIN (
        SELECT DISTINCT
            a.foirequestnumber,
            a.recordspagecount
        FROM
            factFOIFlowRequestStatusDetails AS a
            JOIN (
                SELECT
                    foirequestnumber,
                    MAX(version) AS maxv
                FROM
                    factFOIFlowRequestStatusDetails
                WHERE
                    recordspagecount IS NOT NULL
                    AND isactive = TRUE
                GROUP BY
                    foirequestnumber
            ) AS c ON c.foirequestnumber = a.foirequestnumber
            AND c.maxv = a.version
        WHERE
            a.recordspagecount IS NOT NULL
            AND a.isactive = TRUE
    ) AS frd ON frd.foirequestnumber = rd.visualrequestfilenumber
    LEFT JOIN factRequestPaymentTransaction AS pt ON rd.foirequestid = pt.foirequestid
    AND pt.activeflag = 'Y'
    JOIN (
        SELECT
            MAX(runcycleid) AS maxruncycleid,
            visualrequestfilenumber
        FROM
            factRequestDetails
        WHERE
            activeflag = 'Y'
        GROUP BY
            visualrequestfilenumber
    ) AS maxruncycle ON rd.visualrequestfilenumber = maxruncycle.visualrequestfilenumber
    AND rd.runcycleid = maxruncycle.maxruncycleid
WHERE
    rd.rn = 1
    AND (
        rd.closeddate >= TIMESTAMP('2019-04-01 00:00:00')
        OR rd.closeddate = TIMESTAMP('1900-01-01 00:00:00')
    )
    AND eco.officecode NOT IN ('IAO', 'IMB', 'XGR', 'PROAC');
    -- AND rd.officeid NOT IN ('IAO', 'IMB', 'XGR', 'PROAC');

In [0]:
%sql

-- get data from materialized view
ALTER VIEW default.viw_allopenclosed AS
SELECT
    visualrequestfilenumber AS `Request ID`,
    groupname AS `User Group`,
    ministry AS `Public Body`,
    officecode AS `Office Code`,
    requesttypename AS `Request Type`,
    requestertypename AS `Applicant Category`,
    fullname AS `Applicant Name`,
    subject AS `Subject`,
    try_cast(startdate AS DATE) AS `Start Date`,
    try_cast(targetdate AS DATE) AS `Due Date`,
    try_cast(closeddate AS DATE) AS `Closed Date`,
    processeddays AS `Total Process Days`,
    primaryusername AS `Primary User`,
    requestdescription AS `Description`,
    requeststatusname AS `Request Status`,
    countontime AS `Count On Time`,
    countoverdue AS `Count Overdue`,
    onholddays AS `On Hold Days`,
    daysoverdue AS `Days Overdue`,
    estimatedamount AS `Fees Est`,
    feewaived AS `Fees Waived`,
    feepaidamount AS `Fees Paid`,
    disposition AS `Disposition`,
    publication AS `Publication`,
    publicationreason AS `Publication Reason`,
    numberofextensions AS `Extension`,
    noofpagesintherequest AS `Total AXIS Pages`,
    try_cast(recordspagecount AS BIGINT) AS `Total Mod Pages`,
    isconsultflag,
    isphasedrelease,
    isoipcreview
FROM default.materialized_viw_allopenclosed;